In [2]:
import torch
import torch.nn as nn
from torchvision import transforms, models, datasets

In [3]:
class YOLOv1BackBone(nn.Module):
    def __init__(self, num_classes=100):
        super(YOLOv1BackBone, self).__init__()
        self.features = nn.Sequential(
                            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(7, 7), stride=2, padding=3),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
                            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=(3, 3), stride=1, padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.MaxPool2d(kernel_size=(2, 2), stride=2),

                            nn.Conv2d(in_channels=192, out_channels=128, kernel_size=(1, 1)),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(1, 1)),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.MaxPool2d(kernel_size=(2, 2), stride=2),

                            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=(1, 1)),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=(1, 1)),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=(1, 1)),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=(1, 1)),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(1, 1)),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.MaxPool2d(kernel_size=(2, 2), stride=2),

                            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=(1, 1)),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=(1, 1)),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=(3, 3), stride=2, padding=1),
                            nn.LeakyReLU(negative_slope=0.1),

                            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1),
                            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=(3, 3), padding=1),
                            nn.LeakyReLU(negative_slope=0.1)
                        )

        self.classifier = nn.Sequential(
                        nn.Flatten(),
                        nn.Linear(in_features=1024 * 7 * 7, out_features=4096),
                        nn.Dropout(),
                        nn.LeakyReLU(negative_slope=0.1),
                        nn.Linear(in_features=4096, out_features=num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        # print(x.shape)
        x = self.classifier(x)
        return x

In [30]:
# initial model
model = YOLOv1BackBone(num_classes=100)

# device (gpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)


In [31]:
# define data load
transform = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276]),
])

train_dataset = datasets.CIFAR100(
    root='./data',
    train=True,
    transform=transform,
    download=True,   
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)

Files already downloaded and verified


In [33]:
from tqdm import tqdm
epochs = 30

for epoch in tqdm(range(epochs)):
    model.train()
    loss_train = 0.0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
    
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_train+=loss.item()

    print(f'Epoch:{epoch+1}/{epochs}, Loss:{loss_train/len(train_loader)}')

  3%|███▉                                                                                                                 | 1/30 [05:29<2:39:22, 329.74s/it]

Epoch:1/30, Loss:3.8631150923726505


  7%|███████▊                                                                                                             | 2/30 [11:00<2:34:09, 330.35s/it]

Epoch:2/30, Loss:3.716121257723445


 10%|███████████▋                                                                                                         | 3/30 [16:31<2:28:48, 330.68s/it]

...

Epoch:28/30, Loss:0.20538147534612958


 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 29/30 [2:40:12<05:31, 331.77s/it]

Epoch:29/30, Loss:0.18964939056169194


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [2:45:45<00:00, 331.53s/it]

Epoch:30/30, Loss:0.17744083578229103


In [34]:
torch.save(model.state_dict(), 'yolov1_CIFAR100_pretrain_40epochs.pth')

In [4]:
S = 7
C = 20
B = 2
depth = 5 * B + C
model_yolov1 = YOLOv1BackBone(100)
pretrained_weights = torch.load('yolov1_CIFAR100_pretrain_40epochs.pth')
model_yolov1.load_state_dict(pretrained_weights)

/tmp/ipykernel_3005686/1268867158.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_weights = torch.load('yolov1_CIFAR100_pretrain_40epochs.pth')


<All keys matched successfully>